<a href="https://colab.research.google.com/github/MarylouBer/MLD_Data_Engineering_portfolio/blob/main/Transform_rawJSON_into_SNS_JSON_for_SQS_queues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import uuid
import random
from datetime import datetime

def process_file_based_on_entity(file_path):

    # Step 1: Read the file and parse the JSON content
    with open(file_path, 'r') as file:
        rawjson = json.load(file)

    # Step 2: Check the entity type and call the appropriate function. Each of these types of entity have a different payload structure
    entity = rawjson.get('entity', '')

    if entity in ("XX","YY","ZZ"):
        print("Processing as payload format 1...")
        transform_rawjson_to_snsjson_format1(file_path)
    elif entity in ("AA","BB"):
        print("Processing as payload format 2...")
        transform_rawjson_to_snsjson_format2(file_path)
    else:
        print("Unsupported entity type!")


def transform_rawjson_to_snsjson_format1(file_path):

    # Step 1: Read + Parse the orginal raw json file
    with open(file_path, 'r') as file:
        rawjson = file.read()
        parsedrawjson = json.loads(rawjson) #we also need to parse it to be able use the logic .get

    # Step 3: Build the nested message for XYZ markets
    message = {
        "country_code": parsedrawjson.get("entity", "")[-1:], #we need to add manually the country code as it is not in the raw json payload
        "event_id": str(uuid.uuid4()),
        "data": parsedrawjson, #content of our raw json payload
        "request_id": str(uuid.uuid4()),
        "event_time": datetime.utcnow().isoformat() + "Z",
        "event_time_ms": datetime.utcnow().isoformat() + "Z"
    }

    # Step 4: Build the SNS json
    snsjson = {
        "Type": "Notification",
        "MessageId": str(uuid.uuid4()),
        "TopicArn": "dummy",
        "Message": json.dumps(message),
        "Timestamp": datetime.utcnow().isoformat() + "Z",
        "SignatureVersion": "1",
        "Signature": "dummy-signature",
        "SigningCertURL": "dummy",
        "UnsubscribeURL": "dummy"
    }

    # Step 5: Print nicely
    print(json.dumps(snsjson, indent=2))


def transform_rawjson_to_snsjson_format2(file_path):

    # Step 1: Read the original raw json file
    with open(file_path, 'r') as file:
        rawjson = file.read() #file.read only as we don't need to parse it as we don't use the .get logic here

    # Step 2: Build the SNS json (no need to have a previous step, all needed data is in the rawjson)
    snsjson = {
        "Type": "Notification",
        "MessageId": str(uuid.uuid4()),
        "TopicArn": "dummy",
        "Message": rawjson,
        "Timestamp": datetime.utcnow().isoformat() + "Z",
        "SignatureVersion": "1",
        "Signature": "dummy-signature",
        "SigningCertURL": "dummy",
        "UnsubscribeURL": "dummy"
    }

    # Step 3: Print nicely
    print(json.dumps(snsjson, indent=2))


# --- Run the function with your file ---
process_file_based_on_entity('/Users/xxx/Documents/xxx.json')